<a href="https://colab.research.google.com/github/jcsale/jcsale/blob/main/RDD2020_%2B_YoloV4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://data.mendeley.com/public-files/datasets/5ty2wb6gvg/files/0c2a01c2-b01d-41b4-908e-67fe5bfe7f91/file_downloaded

--2022-03-24 01:47:59--  https://data.mendeley.com/public-files/datasets/5ty2wb6gvg/files/0c2a01c2-b01d-41b4-908e-67fe5bfe7f91/file_downloaded
Resolving data.mendeley.com (data.mendeley.com)... 162.159.130.86, 162.159.133.86
Connecting to data.mendeley.com (data.mendeley.com)|162.159.130.86|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com/23c8e42c-1f62-4804-aeee-15dcf384391f [following]
--2022-03-24 01:48:00--  https://md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com/23c8e42c-1f62-4804-aeee-15dcf384391f
Resolving md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com (md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com)... 52.218.1.32
Connecting to md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com (md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com)|52.218.1.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1472626254 (1.4G) [application/

In [3]:
!tar xvf "/content/file_downloaded"

A saída de streaming foi truncada nas últimas 5000 linhas.
train/Japan/annotations/xmls/Japan_009766.xml
train/Japan/annotations/xmls/Japan_006254.xml
train/Japan/annotations/xmls/Japan_008726.xml
train/Japan/annotations/xmls/Japan_004664.xml
train/Japan/annotations/xmls/Japan_007945.xml
train/Japan/annotations/xmls/Japan_006391.xml
train/Japan/annotations/xmls/Japan_005903.xml
train/Japan/annotations/xmls/Japan_008200.xml
train/Japan/annotations/xmls/Japan_003663.xml
train/Japan/annotations/xmls/Japan_001949.xml
train/Japan/annotations/xmls/Japan_011141.xml
train/Japan/annotations/xmls/Japan_003141.xml
train/Japan/annotations/xmls/Japan_002870.xml
train/Japan/annotations/xmls/Japan_008094.xml
train/Japan/annotations/xmls/Japan_007381.xml
train/Japan/annotations/xmls/Japan_012963.xml
train/Japan/annotations/xmls/Japan_007843.xml
train/Japan/annotations/xmls/Japan_004889.xml
train/Japan/annotations/xmls/Japan_011099.xml
train/Japan/annotations/xmls/Japan_003745.xml
train/Japan/annotatio

In [ ]:
#!mv "/content/train/India" "/content/"

In [ ]:
# import os
# os.rename("/content/India/","/content/Data_India/")

In [15]:
import os, shutil, sys 

def move_to_root_folder(root_path, cur_path):
    for filename in os.listdir(cur_path):
        if os.path.isfile(os.path.join(cur_path, filename)):
            shutil.move(os.path.join(cur_path, filename), os.path.join(root_path, filename))
        elif os.path.isdir(os.path.join(cur_path, filename)):
            move_to_root_folder(root_path, os.path.join(cur_path, filename))
        else:
            sys.exit("Should never reach here.")
    # remove empty folders
    if cur_path != root_path:
        os.rmdir(cur_path)

In [20]:
move_to_root_folder("/content/Data_India", "/content/Data_India/annotations")
move_to_root_folder("/content/Data_India", "/content/Data_India/images")

In [ ]:
import shutil

shutil.rmtree('/content/train')

In [26]:
# Convert PascalVOC Annotations to YOLO


"""
1. The data is taken from https://data.mendeley.com/datasets/5ty2wb6gvg/1
2. RDD2020 dataset comprising 26,336 road images from India, Japan, and the Czech Republic with more than 31,000 instances of road damage
3. Here we consider the images from India alone, to reduce complexity
4. There are four types of road damage: longitudinal cracks (D00), transverse cracks (D10), alligator cracks (D20), and potholes (D40)
5. The data is present in the PascalVOC format as bounding boxes labelled as xmin, ymin, xmax and ymax, stored as xml files
6. This has to converted to fomat for Yolov4 is: object-class x y width height
7. Here object-class is an integer number denoting the class, x y width height - float values relative to width and height of image, it can be equal from (0.0 to 1.0]
"""


import glob
import os
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join


def getImagesInDir(dir_path):
    image_list = []
    for filename in glob.glob(dir_path + '/*.jpg'):
        image_list.append(filename)

    return image_list


def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)


def convert_annotation(dir_path, output_path, image_path):
    basename = os.path.basename(image_path)
    basename_no_ext = os.path.splitext(basename)[0]

    in_file = open(dir_path + '/' + basename_no_ext + '.xml')
    out_file = open(output_path + basename_no_ext + '.txt', 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        try:
            try:
                difficult = obj.find('difficult').text
            except:
                difficult = 0
            cls = obj.find('name').text
            if cls not in classes or int(difficult)==1:
                continue
            cls_id = classes.index(cls)
            xmlbox = obj.find('bndbox')
            b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
            bb = convert((w,h), b)
            out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')
        except:
            pass
            print("Image problem: ", image_path)


dir_path = '/Data_India'
classes = ['D00', 'D10', 'D20', 'D40', 'D44']

def main():
    count = 0
    cwd = getcwd()

    full_dir_path = cwd + '/' + dir_path
    print("full_dir_path: ", full_dir_path)
    output_path = full_dir_path +'/yolo/'
    print("output_path: ", output_path)

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    image_paths = getImagesInDir(full_dir_path)
    list_file = open(full_dir_path + '.txt', 'w')

    for image_path in image_paths:
        list_file.write(image_path + '\n')
        convert_annotation(full_dir_path, output_path, image_path)
        count = count + 1
        print("Image Converted: ", count, ":", image_path)
    list_file.close()

    print("Finished processing: " + dir_path)


if __name__ == '__main__':
    main()

A saída de streaming foi truncada nas últimas 5000 linhas.
Image Converted:  2708 : /content//Data_India/India_006253.jpg
Image Converted:  2709 : /content//Data_India/India_006592.jpg
Image Converted:  2710 : /content//Data_India/India_008109.jpg
Image Converted:  2711 : /content//Data_India/India_002705.jpg
Image Converted:  2712 : /content//Data_India/India_006760.jpg
Image Converted:  2713 : /content//Data_India/India_004942.jpg
Image Converted:  2714 : /content//Data_India/India_001407.jpg
Image Converted:  2715 : /content//Data_India/India_005895.jpg
Image Converted:  2716 : /content//Data_India/India_002047.jpg
Image Converted:  2717 : /content//Data_India/India_006628.jpg
Image Converted:  2718 : /content//Data_India/India_004150.jpg
Image Converted:  2719 : /content//Data_India/India_006758.jpg
Image Converted:  2720 : /content//Data_India/India_003348.jpg
Image Converted:  2721 : /content//Data_India/India_008236.jpg
Image Converted:  2722 : /content//Data_India/India_004833.

In [27]:
move_to_root_folder("/content/Data_India", "/content/Data_India/yolo")

In [38]:
from pathlib import Path

for filename in Path("/content/Data_India").glob("*.xml"):
    filename.unlink()

In [40]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15412, done.
remote: Total 15412 (delta 0), reused 0 (delta 0), pack-reused 15412
Receiving objects: 100% (15412/15412), 14.05 MiB | 18.66 MiB/s, done.
Resolving deltas: 100% (10354/10354), done.
